# Problem Set #7
### MACS 30100 Dr. Evans
### Xiuyuan Zhang

In [41]:
# import library
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from pylab import rcParams
import warnings
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

## Problem 1 Classifier “horse” race
### (a) Logistic Regression

In [42]:
# Import data, drop NA
auto_df = pd.read_csv('Auto.csv',na_values='?')
auto_df.info()
auto_df = auto_df.dropna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
mpg             397 non-null float64
cylinders       397 non-null int64
displacement    397 non-null float64
horsepower      392 non-null float64
weight          397 non-null int64
acceleration    397 non-null float64
year            397 non-null int64
origin          397 non-null int64
name            397 non-null object
dtypes: float64(4), int64(4), object(1)
memory usage: 28.0+ KB


In [43]:
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [44]:
# drop column
auto_df = auto_df.drop(['name'], axis = 1)
auto_df['horsepower'] = auto_df['horsepower'].convert_objects(convert_numeric = True)
auto_df['mpg_high'] = auto_df['mpg'].apply(lambda x: 1 if x >= auto_df['mpg'].median() else 0).astype('category')

In [45]:
# set the variables
X = auto_df[['cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
y = auto_df['mpg_high'].values

In [46]:
# set k-fold cross-validation to k = 4
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(X)
General_error_0 = np.zeros(k)
General_error_1 = np.zeros(k)
General_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    print('When k =', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression(fit_intercept=True) #add constant fit_intercept = True
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE = pd.DataFrame({'Error': (y_test != y_pred).astype(int), 
                        'y_pred': y_pred})
    error_0 = MSE[MSE['y_pred'] == 0]['Error'].mean()
    error_1 = MSE[MSE['y_pred'] == 1]['Error'].mean()
    error_all_class = MSE['Error'].mean()
    General_error_0[k_ind] = error_0
    General_error_1[k_ind] = error_1
    General_MSE[k_ind] = error_all_class
    print(classification_report(y_test, y_pred, digits=3))
    print('The error rate for mpg_high = 0 is', error_0)
    print('The error rate for mpg_high = 1 is', error_1)
    print('The MSE is', error_all_class) 
    print()
    k_ind += 1

print('The k-fold results are as follows:')
print('The average error rate of mpg_high = 0 is', General_error_0.mean())
print('')
print('The average error rate of mpg_high = 1 is', General_error_1.mean())
print('')
print('The average MSE of this model for all classes is', General_MSE.mean())

When k = 0
             precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

The error rate for mpg_high = 0 is 0.0576923076923
The error rate for mpg_high = 1 is 0.130434782609
The MSE is 0.0918367346939

When k = 1
             precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

The error rate for mpg_high = 0 is 0.122448979592
The error rate for mpg_high = 1 is 0.0816326530612
The MSE is 0.102040816327

When k = 2
             precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg / total      0.868     0.867     0.867        98

The error rate for mpg_high = 0 is 0.152173913043
The error rate 

### (b) Random Forest 

In [47]:
random_forest = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                       oob_score=True, random_state=25)
X_rf = auto_df[['cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
random_forest.fit(X_rf, y)
rf_pred = random_forest.oob_decision_function_
MSE_rf = pd.DataFrame({'ypred' : rf_pred.T[1], 'yvals': y})
MSE_rf['ypred'] = MSE_rf['ypred'].apply(lambda x: 1 if x >= 0.5 else 0)

In [48]:
# mean squared error
MSE_model = mean_squared_error(MSE_rf['yvals'], MSE_rf['ypred'])
print('The MSE of the random forest model is:',MSE_model)

# mpg_high = 0
MSE_rf_0 = MSE_rf[MSE_rf['ypred'] < 0.5]
MSE_0 = mean_squared_error(MSE_rf_0['yvals'], MSE_rf_0['ypred'])
print('The MSE of the mpg_high = 0 is:',MSE_0)

# mpg_high = 1
MSE_rf_1 = MSE_rf[MSE_rf['ypred'] >= 0.5]
MSE_1 = mean_squared_error(MSE_rf_1['yvals'], MSE_rf_1['ypred'])
print('The MSE of the mpg_high = 1 is:',MSE_1)

The MSE of the random forest model is: 0.0714285714286
The MSE of the mpg_high = 0 is: 0.0578947368421
The MSE of the mpg_high = 1 is: 0.0841584158416


### (c) SVC (Support Vector Machine)

In [52]:
# set k-fold cross-validation to k = 4
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(X_rf)
SVMGeneral_error_0 = np.zeros(k)
SVMGeneral_error_1 = np.zeros(k)
SVMGeneral_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X_rf):
    print('When k =', k_ind)
    X_train, X_test = X_rf[train_index], X_rf[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Set the penalty parameter to C=1 and set gamma=0.2
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum()
    SVMGeneral_error_0[k_ind] = error_0
    SVMGeneral_error_1[k_ind] = error_1
    SVMGeneral_MSE[k_ind] = error_all_class
    print(classification_report(y_test, y_pred, digits=3))  
    print('The error rate of mpg_high = 0 is', error_0)
    print('The error rate of mpg_high = 1 is', error_1)
    print('The MSE is', error_all_class) 
    print()
    k_ind += 1

When k = 0
             precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

The error rate of mpg_high = 0 is 0.0
The error rate of mpg_high = 1 is 0.552083333333
The MSE is 0.540816326531

When k = 1
             precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

The error rate of mpg_high = 0 is 0.520408163265
The error rate of mpg_high = 1 is nan
The MSE is 0.520408163265

When k = 2
             precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

The error rate of mpg_high = 0 is 0.53125
The error rate of mpg_high = 1 is 0.0
The MSE is 0.5

In [53]:
SVMGeneral_error_0 = SVMGeneral_error_0[~np.isnan(SVMGeneral_error_0)]
print('The average error rate of mpg_high = 0 is', SVMGeneral_error_0.mean())
print()
SVMGeneral_error_1 = SVMGeneral_error_1[~np.isnan(SVMGeneral_error_1)]
print('The average error rate of mpg_high = 1 is', SVMGeneral_error_1.mean())
print()
print('The average MSE of the SVM model is', SVMGeneral_MSE.mean())

The average error rate of mpg_high = 0 is 0.381194110709

The average error rate of mpg_high = 1 is 0.184027777778

The average MSE of the SVM model is 0.507653061224


### (d) Which of the above three models do you think is the best predictor of mpg high? Why?

Looking at the results of average model MSE, average error rate of each category from the previous three models:

#### For the Logistic regression model:

The average error rate of mpg_high = 0 is 0.0947067070586

The average error rate of mpg_high = 1 is 0.118226649127

The average MSE of the logistic regression model is 0.107142857143

#### For the random forest model:

The average error rate of mpg_high = 0 is: 0.0578947368421

The average error rate of mpg_high = 1 is: 0.0841584158416

The average MSE of the random forest model is: 0.0714285714286

#### For the SVM model:

The average error rate of mpg_high = 0 is 0.381194110709

The average error rate of mpg_high = 1 is 0.184027777778

The average MSE of the SVM model is 0.507653061224

One can see that the Randome Forest model has the least amount of average error rate for both categories within the class mpg_high, and an overall least average MSE, among all three models. It is clear from our understanding about fitting models and performing predictions that the smaller the error rate and MSE the more accurate the prediction results. Thus, I think that the Random Forest model is the best predictior of $mpg_high$.

One should also note the different methods used to compute MSE values in these models. While both SVC and Logistic Regression used k-fold cross validation, the Random Forest model used OOB prediction function. Using OOB prediction function when the sample size is not sufficiently large could lead to smaller MSE values comparing to the cross validation method with the same sample size. It is thus more advantageous to standardize the methologies when possible. Here, however, one can still use not only the average MSE values but the average error rate to get a comparison that overall the Random Forest model achieved the best performance.